# Example auto-grading
The code below gathers students' poll answers from session 10.2 of Minerva University Formal Analysis class using BeautifulSoup.
I first clean the data by dropping rows where student did not write an answer.  


The question of the poll was "What is the likelihood ratio, P(S|G)/P(S|~G), and what would it mean for this scenario?"
Sample answer: "The likelihood ratio is P(R|M)/P(R|~M) = 0.7/0.5 = 1.4 < 1. We know that (posterior odds) = (likelihood ratio) * (prior odds), so the fact that the likelihood ratio is less than 1 means that the odds on the posterior hypothesis (that the maid is the murderer) is less than the odds on the prior hypothesis. The evidence (rope) is not in support of convicting the maid, but rather supports her innocence. This likelihood ratio is useful, but to really assess how probable it is that the maid is guilty, we would really want to compute the odds on the posterior."

Rubric:
2:
    -Incorrect calculation (give a 1 if it’s really way off with no explanation given)
    -Or, correct calculation but nothing in the interpretation is right.
3: 
    -Correct calculation and some part of the interpretation is correct, but maybe not all of the interpretation was right, or maybe there was insufficient explanation for how the calculation was done
4:
    -The calculation is correct, steps (formulas) are clearly indicated, and the interpretation makes complete sense




(machine learning result + rubric score)/2

In [1]:
from bs4 import BeautifulSoup
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer


tokenizer = RegexpTokenizer(r'\w+')
column_titles = ['student name', 'response']
answers = []

with open('home.html','r') as html_file:
    content = html_file.read()
    soup= BeautifulSoup(content,'lxml')
    #print(soup.prettify())
    polls_html_tags = soup.find_all('pre')
    names = soup.find_all('p')
    for i in range(len(names)):
        #filters out punctuations and separates a sentence into words
        #filtered_answer = tokenizer.tokenize(polls_html_tags[i].text)
        answers.append([names[i].text,polls_html_tags[i].text])
    # for name in names:
    #     print(name.text)
    # for poll in polls_html_tags:
    #     answers.append(poll.text)
df=pd.DataFrame(columns=column_titles, data = answers)
indexNames = df[ df['response'] == 'No response submitted.' ].index
df.drop(indexNames , inplace=False)
scores= []
# for i in range(len(names)):
#     scores.append()
df

,student name,response
0,Ahmed Ibrahim,"Here, the evidence is that the shovel is a pos..."
1,Cynthia Peng,P(G) = 0.2\n\nlikelihood = P(S|G) / P(S | ~G) ...
2,Emil Rustamli,LR = 0.7/0.5 = 1.4\nThis means that it is 40% ...
3,Helen Yuya,P(S|G)/P(S|~G) = 0.7/0.5 = 1.4\nSince the like...
4,Ibrahim Jehangir,Prior H:\nGK is murderer P(G) = 0.2\nGK isnt m...
5,Iram Ikbal Inamdar,No response submitted.
6,Keita Ueno,No response submitted.
7,Khaled Ahmed,No response submitted.
8,Khas Bayarjargal,No response submitted.
9,Lisa Pishina,No response submitted.


In [119]:
# numeric answer to the question
numeric_answer=['1.4']

# criteria 1
keywords1 = ['0.7/0.5','0.7 / 0.5']

# criteria 2
keywords2 = ['odds','bigger','higher','more']

# criteria 3
keywords3 = ['groundkeeper','shovel', 'support','murderer']

# negative keywords: keywords that lead to wrong interpretation 
n_keywords = ['%','chance']

keywords = [keywords1, keywords2, keywords3]

# positive comments for answers that include the keywords
p_comment1="Great job with your calculation!"
p_comment2="I like your interpretation of the likelihood ratio!"
p_comment3="Keep in mind that while this likelihood ratio is useful, to really assess how probable it is that the maid is guilty, we would really want to compute the odds on the posterior. That's where Bayes' formula comes in."
p_comments=[p_comment1,p_comment2,p_comment3]

# comments for answers that does not include the keywords
comment1="I wish you showed the steps of the calculation."
comment2="So the fact that the likelihood ratio is bigger than 1 means that the odds on the posterior hypothesis (that the groundkeeper is the murderer) is bigger than the odds on the prior hypothesis."
comment3= "I wish it was more applied to the given scenario. What is the 'hypothesis' that you are referring to here? Let's connect it to the groundkeeper and the shovel."
comments = [comment1, comment2, comment3]

# comment for answers with negative keywords
n_comment1="It's important to distinguish between probabilities and odds. This is not a probability, so should not be referred to a % or chance. Instead, it's a ratio of probabilities. It tells us how one probability compares to another."
n_comments = [n_comment1]
scores=[]



def calculation_check(answer):
    """
    Input: str
        -student poll answer
    Output: Bool

    Searches for the numeric answer in the input string. 
    Returns True if the numeric answer is found, and returns False otherwise.
    """
    for keyword in keywords1:
        if keyword in answer:
            return True
    return False

def poll_grader():
    """
    Calculates 
    """
    for i in range(len(df)):
        Score = 0
        criteria=0
        criteria_score=0
        name = df['student name'][i]
        final_comment=[]
        response = df['response'][i]
        if numeric_answer:
            if calculation_check(response.split())==True:
                Score+=3
        for j in range(len(keywords)):
            if keywords[j]:
                criteria+=1
                found = False
                for keyword in keywords[j]:
                    if keyword in response:
                        criteria_score +=1
                        found = True
                        final_comment.append(p_comments[j])
                        break
                if found==False:
                    criteria_score-=1
                    final_comment.append(comments[j])
        if n_keywords[0]:
            for n_keyword in n_keywords:
                if n_keyword in response:
                    final_comment.append(n_comments[0])
                    criteria_score -= 2
        Score += criteria_score/criteria
        print(name, Score)
        for i in final_comment:
            print(i)
        

In [3]:
class Poll_Grader:
    
    def __init__ (self, name, response):
        self.name = name
        self.response = response

    def calculation_checks(self, answer):
        """
        Input: str
            -student poll answer
        Output: Bool

        Searches for the numeric answer in the input string. 
        Returns True if the numeric answer is found, and returns False otherwise.
        """
        for keyword in keywords1:
            if keyword in answer:
                return True
        return False

    def poll_grader(self):
        """
        
        """
        for i in range(len(df)):
            Score = 0
            criteria=0
            criteria_score=0
            name = df['student name'][i]
            final_comment=[]
            response = df['response'][i]
            if numeric_answer:
                if calculation_check(response.split())==True:
                    Score+=3
            for j in range(len(keywords)):
                if keywords[j]:
                    criteria+=1
                    found = False
                    for keyword in keywords[j]:
                        if keyword in response:
                            criteria_score +=1
                            found = True
                            final_comment.append(p_comments[j])
                            break
                    if found==False:
                        criteria_score-=1
                        final_comment.append(comments[j])
            if n_keywords[0]:
                for n_keyword in n_keywords:
                    if n_keyword in response:
                        final_comment.append(n_comments[0])
                        criteria_score -= 2
            Score += criteria_score/criteria
            print(name, Score)
            for i in final_comment:
                print(i)

IndentationError: expected an indented block (755424790.py, line 7)

In [120]:
poll_grader()

Ahmed Ibrahim 4.0
Great job with your calculation!
I like your interpretation of the likelihood ratio!
Keep in mind that while this likelihood ratio is useful, to really assess how probable it is that the maid is guilty, we would really want to compute the odds on the posterior. That's where Bayes' formula comes in.
Cynthia Peng 3.3333333333333335
Great job with your calculation!
I like your interpretation of the likelihood ratio!
Keep in mind that while this likelihood ratio is useful, to really assess how probable it is that the maid is guilty, we would really want to compute the odds on the posterior. That's where Bayes' formula comes in.
It's important to distinguish between probabilities and odds. This is not a probability, so should not be referred to a % or chance. Instead, it's a ratio of probabilities. It tells us how one probability compares to another.
Emil Rustamli 2.6666666666666665
Great job with your calculation!
I like your interpretation of the likelihood ratio!
I wish

In [77]:
df['student name'][3]

'Helen Yuya'

In [45]:
for i in range(len(df['response'])):
        Score = 0
        criteria=0
        criteria_score=0
        final_comment=[]
        response = df['response'][i].split()
        print(response)

['Here,', 'the', 'evidence', 'is', 'that', 'the', 'shovel', 'is', 'a', 'possible', 'murder', 'weapon', '(S).', 'The', 'hypothesis', 'here', 'is', 'that', 'the', 'groundkeeper', 'is', 'the', 'murderer.', 'The', 'likelihood', 'is', 'P(S|G)/P(S|~G)', '=', '0.7/0.5', '=', '1.4.', 'The', 'likelihood', 'is', '>', '1.', 'This', 'means', 'that', 'the', 'probability', 'that', 'the', 'evidence', 'was', 'true', '(the', 'shovel', 'was', 'used)', 'given', 'that', 'the', 'hypothesis', 'is', 'true', '(groundkeeper', 'is', 'the', 'murderer)', 'is', 'more', 'than', 'the', 'probability', 'that', 'the', 'evidence', 'is', 'true', '(the', 'shovel', 'was', 'used)', 'given', 'that', 'the', 'hypothesis', 'is', 'false', '(groundkeeper', "isn't", 'the', 'murderer).', 'As', 'a', 'result,', 'the', 'evidence', 'is', 'favoring', 'the', 'hypothesis', 'is', 'true', '(groundkeeper', 'is', 'the', 'murderer).']
['P(G)', '=', '0.2', 'likelihood', '=', 'P(S|G)', '/', 'P(S', '|', '~G)', '=', '0.7/0.5', '=', '1.4', '1.4', '

KeyError: 5

In [50]:
df['response'][5].split()

KeyError: 5

In [121]:
class Poll:
    def __init__(self, name, answer):
        self.name = name
        self.answer = answer

class poll_grader:
    def __init__(self, polls):
        self.polls = polls

    def calculation_check(answers):
        for i in range(len(answers)):
            for keyword in keywords1:
                if keyword in answers[i][1]:
                    return True
            return False